In [1]:
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import pickle
import random
import numpy as np
import timeit
from collections import defaultdict, Counter
from pathlib import Path
from Bio import SeqIO
import sys
lib_dir = '../mylibs'
if lib_dir not in sys.path:
    sys.path.append(lib_dir)
    
import vhdb as vhdb
import features_predict as fp

In [2]:
filein = '../inputs/VHDB_25_1_2019.p'
with open(filein, 'rb') as f:
    V_H = pickle.load( f)
hosts = V_H.hosts
viruses = V_H.viruses
subspecies =V_H.subspecies

In [10]:

# Input file
#subsets = [(label,taxlevel,pool,taxlevelpool),...]
subsetfile = '../inputs/Euk_RNA.csv'
baltimore = 'all'
vhdbfile = '../inputs/VHDB_25_1_2019.p'


# Everthing in the same order as the features list

features = ['DNA','AA','PC','Domains']
# filepath for the fasta, faa and domain files
filepaths = [
                '/home4/youn01f/Desktop/workspace/newData/fasta',
                '/home4/youn01f/Desktop/workspace/newData/faa','/home4/youn01f/Desktop/workspace/newData/faa',
                '/home4/youn01f/Desktop/workspace/newData/pfs']
file_exts = ['fasta','fasta','fasta','pfs']#,'fasta',fasta]

# A list of kmers for each feature set to be tested
kmer_lists = [[1, 2,3,4,5,6,7,8,9], # dna 
              [1, 2, 3,4], # aa
              [1, 2,3,4,5,6] ,    #pc
             [0]] #domains

# symbol dictionaries 
na_dict = {'mod':4,'a':0,'c':1,'g':2,'t':3}
aa_dict = {'mod':20 ,'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,
              'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}
pc_dict = {'mod':7, 'C':0,
            'A':1,'G':1,'V':1,
           'I':2,'L':2,'F':2,'P':2,
           'M':3,'S':3,'T':3,'Y':3,
           'H':4,'N':4,'Q':4,'W':4,
           'R':5,'K':5,
           'D':6,'E':6}
symbol_dicts = [ na_dict,aa_dict,pc_dict, {}]

classifiers = ['SVM_lin']

# Output file for the results

results_file = f'{Path(subsetfile).stem}_results.csv'
print( f'Results in {results_file}')

Results in Euk_RNA_results.csv


In [4]:
label_info = pd.read_csv(subsetfile)
subsets = label_info.apply(tuple, axis=1).tolist()
len(subsets)

51

In [6]:
for subset in subsets:
    print (subset)
    class_data,n = fp.get_class_lists(subset,viruses,hosts)
    datasets = fp.split_data(viruses,class_data,n)
    (label,label_tax,pool,pool_tax,baltimore) = subset

('Bos', 'genus', 'Bovidae', 'family', 'RNA')
 size of training 172 and test set   40
('Ovis aries', 'species', 'Bovidae', 'family', 'RNA')
 size of training 42 and test set   14
('Bovidae', 'family', 'Mammalia', 'class', 'RNA')
 size of training 224 and test set   58
('Bos taurus', 'species', 'Bovidae', 'family', 'RNA')
 size of training 161 and test set   39
('Sus scrofa', 'species', 'Mammalia', 'class', 'RNA')
 size of training 169 and test set   39
('Suidae', 'family', 'Mammalia', 'class', 'RNA')
 size of training 173 and test set   41
('Vespertilionidae', 'family', 'Chiroptera', 'order', 'RNA')
 size of training 56 and test set   18
('Pteropodidae', 'family', 'Chiroptera', 'order', 'RNA')
 size of training 68 and test set   20
('Chiroptera', 'order', 'Mammalia', 'class', 'RNA')
 size of training 250 and test set   64
('Equus', 'genus', 'Mammalia', 'class', 'RNA')
 size of training 44 and test set   16
('Equidae', 'family', 'Mammalia', 'class', 'RNA')
 size of training 46 and test s

In [ ]:
min(500,250,251)

In [11]:
def extract_kmers (sequences,k,symbol_dict):
    if k == 0: # domainsdom_list = list({dom for s in sequence.values() for dom in s})
        
        X,seq_index,f_index = get_domains(sequences,symbol_dict)
    else:  # everything else
        vocab_len =  symbol_dict['mod'] **k
        f_index = get_feature_names(symbol_dict,k)
        seq_index = []
        X = np.zeros((len(sequences) ,vocab_len))

        for i,(accn, seq) in enumerate (sequences.items()):
            X[i] = get_word_frequ (seq,k, vocab_len,symbol_dict)
            seq_index.append(accn)
# normalise for length of genome(s)
    XN = np.divide(X,X.sum(axis = 1)[:,None])
    print (np.shape(XN)) 
    return XN,seq_index,f_index

def get_domains(sequence,dom_list):
    """input: sequences dictionary of training and test sequence dicts"""
    seq_index = []
    X = np.zeros((len(sequence) ,len(dom_list)))
    for i,(accn, seq) in enumerate (sequence.items()):
        for dom in seq:
            j = dom_list.index(dom)
            X[i,j] += 1
        seq_index.append(accn)
    return X,seq_index,dom_list

def get_word_frequ(sequence,k,vocab_len,symbol_dict):
    """returns the word frequency array for all genomes for a virus """
    word_freqs = np.zeros(shape =(vocab_len),dtype=np.int32)
    for i in range(len(sequence)-k+1):
        word = sequence[i:i+k]
        if check_word(word,symbol_dict):
            kmer_index = patternToNumber (word,k, symbol_dict)
            word_freqs [ kmer_index] += 1
            # If this for DNA/RNA then add the k-mers complinent
#             if symbol_dict['mod'] == 4:
#                 word_freqs [patternToNumber(compliment(word),k,symbol_dict)] +=1   
    return word_freqs

def get_sequences(faapath,datasets,ext):
    missing_files = []
    v_with_missing_files = []
    ext = f'.{ext}'
    sequences = {'training':{},'test':{}}
    for key,dataset in datasets.items():
        for v,vdict in dataset.items():
            sequence = defaultdict(str)
            for ref in vdict['refseqs']:
                filename = os.path.join(faapath, ref.strip()+ ext)
                try:
                    with open(filename, 'rt') as handle:
                        if ext == '.fasta':
                            for record in SeqIO.parse(handle, "fasta"):                   
                                sequence[v] += str(record.seq)
                        else: # pfs file - list of domains  
                            line = handle.read()
                            sequence[v] = [d for d in line.split()]
                                
                except IOError:
                    missing_files.append((v,ref))
                    
            if len(sequence) > 0:
                sequences[key].update(sequence)
            else:
                v_with_missing_files.append(v)
    print ( f'viruses no files {len(set(v_with_missing_files))}')
    return (sequences)


def symbolToNumber (symbol,symbols):
    return symbols[symbol]

def patternToNumber (pattern,k,symbol_dict):
    mod = symbol_dict['mod']
    number = 0
    for i in range (0,k):
        n = symbolToNumber ( pattern [k-1-i],symbol_dict)
        number += n* mod**i 
    return number

def get_feature_names(symbols,k):
    all_kmers = []
    n2sym = { v:k for k,v in symbols.items()}
    mod = symbols['mod']
    for number in range(mod**k):
        pattern = ''
        for i in range (0,k):
            n = number% mod
            number = (number - n)/mod 
            symbol = n2sym[n]
            pattern = symbol + pattern
        all_kmers.append (pattern)
    return all_kmers

def check_word(mystring,symbols):
    """Check for illegal characters """
    return all(c in symbols for c in mystring)

def compliment(dna):
    """Return the reverse compliment of a sequence """
    bp = { 'a':'t', 't':'a', 'c':'g','g':'c'}
    revdna =''    
    for i in range(len(dna)-1,-1,-1):
         revdna += bp[dna[i]]
    return (revdna)

def  get_labels(label,v_index,dataset):      
    for i,v in enumerate(v_index):
        y = [1 if dataset[v]['label'] ==  label else 0 for v in v_index ] 
    return np.asarray (y)

def get_feature_matrices(sequences,datasets,k,symbol_dict):
   
    if k == 0: # domainsdom_list = list({dom for s in sequence.values() for dom in s})
        symbol_dict = list({dom for sequence in sequences.values() for s in sequence.values() for dom in s}) 
    print (f'X_training:Extacting  {feature}  of length  {k}  from   {len(sequences["training"])} sequences')
    X_train, seq_index, f_index = extract_kmers (sequences['training'],k,symbol_dict)
    Y_train =  get_labels (label,seq_index,datasets['training'])
    print (f'X_test: Extacting  {feature}  of length  {k}  from   {len(sequences["test"])} sequences')
    X_test, seq_index, f_index = extract_kmers (sequences['test'],k,symbol_dict)
    Y_test =  get_labels (label,seq_index,datasets['test'])
           
    return X_train,X_test,Y_train,Y_test

## Main Loop

In [ ]:
for subset in subsets:
    class_data,n = fp.get_class_lists(subset,viruses,hosts)
    datasets = fp.split_data(viruses,class_data,n)
    (label,label_tax,pool,pool_tax,baltimore) = subset
    print  (label,label_tax,pool,pool_tax,baltimore)
   
    for feature in feature_list:
        
        index = features.index(feature)
        kmers = kmer_lists[index]
        filepath = filepaths[index]
        symbol_dict = symbol_dicts[index]
        ext = file_exts[index]
        print (index,kmers,filepath,symbol_dict)

        if features != 'PC': #PC same sequence as for AA
            filepath = filepaths[index]
            print ( f'getting {feature} sequences from {filepath}')
            sequences = get_sequences(filepath,datasets,ext)

        for k in kmers: 
            x_train,x_test,y_train,y_test = get_feature_matrices(sequences,datasets,k,symbol_dict)
            print (np.shape(x_train), np.shape(x_test),np.shape(y_train),np.shape(y_test))
            #print (type(x_train), type(x_test),type(y_train),type(y_test))
            results = fp.test_prediction(x_train,x_test,y_train,y_test)
            results.update ({'N in class': n, 'Features':feature, 'k':k})
            print(results)
            fp.results2CSV (results,subset, results_file)

 size of training 172 and test set   40
Bos genus Bovidae family RNA
0 [1, 2, 3, 4, 5, 6, 7, 8, 9] /home4/youn01f/Desktop/workspace/newData/fasta {'mod': 4, 'a': 0, 'c': 1, 'g': 2, 't': 3}
getting DNA sequences from /home4/youn01f/Desktop/workspace/newData/fasta
viruses no files 3
X_training:Extacting  DNA  of length  1  from   170 sequences
(170, 4)
X_test: Extacting  DNA  of length  1  from   39 sequences
(39, 4)
(170, 4) (39, 4) (170,) (39,)
{'AUC': 0.534, 'N in class': 106, 'Features': 'DNA', 'k': 1}
X_training:Extacting  DNA  of length  2  from   170 sequences
(170, 16)
X_test: Extacting  DNA  of length  2  from   39 sequences
(39, 16)
(170, 16) (39, 16) (170,) (39,)
{'AUC': 0.675, 'N in class': 106, 'Features': 'DNA', 'k': 2}
X_training:Extacting  DNA  of length  3  from   170 sequences
(170, 64)
X_test: Extacting  DNA  of length  3  from   39 sequences
(39, 64)
(170, 64) (39, 64) (170,) (39,)
{'AUC': 0.817, 'N in class': 106, 'Features': 'DNA', 'k': 3}
X_training:Extacting  DNA 

### Test one subset on on feature type

In [ ]:
len(datasets['training'])

In [ ]:
feature_list = ['DNA','AA','PC','Domains']
features = ['AA']
for feature in features:
    index = feature_list.index(feature)
    kmers = kmer_lists[index]
    filepath = filepaths[index]
    symbol_dict = symbol_dicts[index]
    ext = file_exts[index]
    print (index,kmers,filepath,symbol_dict)
    
    if features != 'PC': #PC same sequence as for AA
        filepath = filepaths[index]
        print ( f'getting {feature} sequences from {filepath}')
        sequences = get_sequences(filepath,datasets,ext)
    
    for k in kmers: 
        start_time = timeit.default_timer()
        x_train,x_test,y_train,y_test = get_feature_matrices(sequences,datasets,k,symbol_dict)
        print ('timeit: ',timeit.default_timer() - start_time)
        print (np.shape(x_train), np.shape(x_test),np.shape(y_train),np.shape(y_test))
        results = fp.test_prediction(x_train,x_test,y_train,y_test)
        results.update ({'N in class': n, 'Features':feature, 'k':k})
        print(results)
        fp.results2CSV (results,subset, results_file)


In [ ]:
subset
